# Mein Projekt

Erkenntnisse
-----
- Abstand zwischen zwei Punkten im Raster ~48 (da dst(ox, origin) & dst(oy, origin) ~144)
- Abstand zwischen zwei Punkten im Raster, diagonal = sqrt(48^2 + 48^2) ~ 68, mit Messwerten verifizierbar
- Abweichung von theoretischer zu echter Distanz o,x bzw. o,y ~10%

ToDo
----
- test for different touchcodes

In [49]:
samples = {}
samples[0b10000000] = [[(1643,614),(1450,649),(1555,663),(1568,731)],
           [(577,629),(379,577),(471,631),(453,701)],
           [(1486,68),(1638,213), (1581,119),(1628,69)],
           [(1676,651),(1530,799),(1619,750),(1675,791)],
           [(176,469), (320,324),(225,373),(175,324)],            
           [(725,544),(860,697),(819,599),(869,557)],
           [(346,509),(494,368),(399,415),(346,367)],
           [(769,593),(750,792),(793,695),(865,701)],
            [(269,202),(477,205),(375,164),(382,93)]]
                       
garbage = [[(303,152), (379,577),(368,171),(368,285)]]


In [134]:
import itertools
import numpy as np
import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.spatial import distance

def sample_to_ndarray(sample):
    points = []
    
    for point in sample:
        points.append(point[0])
        points.append(point[1])
    
    return np.array(points)


def render_sample(index):
    df = pd.DataFrame(samples[index], columns=['x', 'y'])
    print(df)
    fig = df.plot.scatter(x='x', y='y', color='red', figsize=(15,15))

    fig.set_xlim([0, 2000])
    fig.set_ylim([0, 2000])

    plt.gca().set_aspect('equal', adjustable='box')
    plt.ylabel('some numbers')

    plt.show()

def convex_hull(sample):
    # convert points of sample to ndarray
    points = np.asarray(sample)
    
    # find the convex hull
    hull = ConvexHull(points)    
   
    plt.figure(num=None, figsize=(18, 16), dpi=320, facecolor='w', edgecolor='r')
    
    plt.xlim([0,2000]) 
    plt.ylim([0,2000]) 

    # plot the original points
    plt.plot(points[:, 0], points[:, 1], 'o')

    # plot the convex hull around it
    for simplex in hull.simplices:
        plt.plot(points[simplex, 0], points[simplex, 1], 'r')

    # adjustment to coordinate system
    plt.gca().set_aspect('equal', adjustable='box')
    
    # display that crap
    plt.show()


def get_distances_for_sample(sample):
    oy = sample[0]
    ox = sample[1]
    tc = sample[2] # part of the code
    origin = sample[3] # origin

    return[distance.euclidean(origin, ox), distance.euclidean(origin, oy),distance.euclidean(ox, oy),distance.euclidean(tc, origin) ]
    

def get_statistics_for_sampleset(sampleset):
    dst_origin_ox = []
    dst_origin_oy = []
    dst_ox_oy = []
    dst_origin_tc = []

    for i in range(0,9): 
        sample = samples[0b10000000][i]
        distances = get_distances_for_sample(sample)
        
        dst_origin_ox.append(distances[0])
        dst_origin_oy.append(distances[1])
        dst_ox_oy.append(distances[2])
        dst_origin_tc.append(distances[3])

    print("dst(origin,x): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_ox), np.max(dst_origin_ox), np.min(dst_origin_ox)))
    print("dst(origin,y): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_oy), np.max(dst_origin_oy), np.min(dst_origin_oy)))
    print("dst(ox,oy): mean({0}), max({1}, min({2}))".format(np.mean(dst_ox_oy), np.max(dst_ox_oy), np.min(dst_ox_oy)))
    print("dst(origin,tc): mean({0}), max({1}, min({2}))".format(np.mean(dst_origin_tc), np.max(dst_origin_tc), np.min(dst_origin_tc)))

    
sample = samples[0x80][3]
get_orientation_marks(sample)
#convex_hull(sample)
#get_statistics_for_sampleset(samples[0x80])

def norm_wolf(sample):
    o = np.array(sample[3])
    x = np.array(sample[1])
    y = np.array(sample[0])
    s = np.array(sample[2])
    
    # Richtungsvektoren entlang der Kanten vom Referenzsystem
    vx = x - o
    vy = y - o
    # Ortsvektor des Punkts bzgl o (wo ist s bgzl des neuen Ursprungs o)
    so = s-o
    
    # Normierung der Richtungsvektoren    
    vx = (vx/(np.linalg.norm(vx)))/(np.linalg.norm(vx))*3
    vy = (vy/(np.linalg.norm(vy)))/(np.linalg.norm(vy))*3
   
    xcor = np.dot(vx,so)
    ycor = np.dot(vy,so)
    
    
    dst_x_y = distance.euclidean(x,y)
    dst_o_xy = dst_x_y / np.sqrt(2)
    dst_o_x = distance.euclidean(o,x)
    dst_o_y = distance.euclidean(o,y)
    
    
    print("s.x: {0}, s.y: {1}".format(xcor, ycor))
    print("next step. dst(x,y) = {0} => dst(o,x|y) ~= {1} [is {2} and {3}]".format(dst_x_y, dst_o_xy, dst_o_x, dst_o_y))

    
for sample in samples[0x80]:    
    normn(get_orientation_marks(sample), sample[2])



s.x: 1.033029833397908, s.y: 1.0843429636533086
s.x: 1.0, s.y: 1.1
True
True
s.x: 1.0571647803568003, s.y: 1.0610894941634241
s.x: 1.1, s.y: 1.1
True
True
s.x: 0.9689959685160301, s.y: 0.985469873543268
s.x: 1.0, s.y: 1.0
True
True
s.x: 1.1084451609843993, s.y: 0.8699556145094638
s.x: 1.1, s.y: 0.9
True
True
s.x: 1.0344827586206897, s.y: 1.020878911823457
s.x: 1.0, s.y: 1.0
True
True
s.x: 0.9648899954270616, s.y: 0.9548911743602012
s.x: 1.0, s.y: 1.0
True
True
s.x: 1.0808491212052045, s.y: 1.0140845070422535
s.x: 1.1, s.y: 1.0
True
True
s.x: 1.0788617130103229, s.y: 1.0862068965517244
s.x: 1.1, s.y: 1.1
True
True
s.x: 1.0135379479808986, s.y: 1.0381338742393509
s.x: 1.0, s.y: 1.0
True
True


In [220]:
"""API Cell"""
import itertools
import numpy as np
from operator import itemgetter
from scipy.spatial import ConvexHull
from scipy.spatial import distance
import collections
import pandas as pd

DEBUG = False

def log(message):
    if DEBUG:
        print(message)

def are_same(reference, value, percentage):
    min_value = reference - reference * percentage
    max_value = reference + reference * percentage
    result = min_value < value < max_value
    #print("reference: {0}, min: {1}, max: {2}, value: {3}, matches: {4}".format(reference, min_value, max_value, value, result))
    return result

def norm(reference, point):
    o = reference[0]
    x = reference[1]
    y = reference[2]
    s = point
    
    # Richtungsvektoren entlang der Kanten vom Referenzsystem
    vx = x - o
    vy = y - o
    # Ortsvektor des Punkts bzgl o (wo ist s bgzl des neuen Ursprungs o)
    so = s-o
    
    # Normierung der Richtungsvektoren    
    vx = (vx/(np.linalg.norm(vx)))/(np.linalg.norm(vx))*3
    vy = (vy/(np.linalg.norm(vy)))/(np.linalg.norm(vy))*3
   
    xcor = np.dot(vx,so)
    ycor = np.dot(vy,so)
    
    dst_x_y = distance.euclidean(x,y)
    dst_o_xy = dst_x_y / np.sqrt(2)
    dst_o_x = distance.euclidean(o,x)
    dst_o_y = distance.euclidean(o,y)
    
    log("s.x: {0}, s.y: {1}".format(xcor, ycor))
    log("s.x: {0}, s.y: {1}".format(round(xcor, 1), round(ycor, 1)))
    
    return (round(xcor, 1), round(ycor, 1))



def touchcode_from_points(points):    
    touchcode = 0
    
    for point in points:
        if 0.9 <= point[0] <= 1.1 and 0.9 <= point[1] <= 1.1:
            touchcode |= 0x80
            
            
        if 1
            
    return touchcode

def get_orientation_marks(points):
    p_threshold = 0.08
    no_result = (None, None, None)

    # no touchcode if there are not enough points
    if points is None or len(points) < 3:
        return no_result

    # create a convex hull with all points
    try:
        hull = ConvexHull(points)  
    except:
        return no_result
    
    vectors = []
    
    # calculate all possible distances between all points
    for combination in list(itertools.combinations(hull.vertices, 2)):
        p1 = points[combination[0]]
        p2 = points[combination[1]]
        dst = distance.euclidean(p1, p2)
        
        vectors.append((p1, p2, dst))      
        
    # get the two points that have the longest distance (those are vx and vy)
    orientation = max(vectors, key=itemgetter(2))
    v1 = orientation[0]
    v2 = orientation[1]
    
    log("v1: {0}, v2: {1}, dst(v1, v2): {2}]".format(v1, v2, orientation[2]))
    
    # filter distances list to not contain the longest distance anymore
    vectors = [a for a in vectors if not orientation[2] == a[2]]
    
    origin = None
    candidates = []
    
    # find the origin candidates by getting all distances that are longest_distance / sqrt(2)    
    for vector in vectors:
        if are_same(orientation[2] / np.sqrt(2), vector[2], p_threshold):
            if np.array_equal(vector[0], v1) or np.array_equal(vector[0], v2):
                candidates.append((vector[1][0], vector[1][1]))
            if np.array_equal(vector[1], v1) or np.array_equal(vector[1], v2):
                candidates.append((vector[0][0], vector[0][1]))
    
    # find the origin (the point that we got twice)
    try:
        origin = np.array([k for k, v in collections.Counter(candidates).items() if v == 2])[0]
    except:
        return (None, None, None)
    
    log("origin: {0}".format(origin))
    
    vx, vy = find_vx_vy(hull, origin, v1, v2)
    
    log("vx: {0}, vy: {1}".format(vx, vy))
    
    return (origin, vx, vy)

def find_vx_vy(hull, origin, v1, v2):    
    # find the index of the origin in the points list
    origin_index = 0
    for v in hull.vertices:
        if np.array_equal(hull.points[v], origin):
            origin_index = v
    
    # roll the vertices to start with the origin. vx will always be to the right of the origin, vy will be left
    verts = np.roll(hull.vertices, -np.asscalar(np.where(hull.vertices == origin_index)[0]))

    for v in verts:
        if np.array_equal(hull.points[v], v1):
            return v1, v2
            
        if np.array_equal(hull.points[v], v2):
            return v2, v1
            vx = v2
    

In [223]:
import unittest
from unittest import *

class TestPointsToTouchcode(TestCase):
    def test_no_points(self):
        points = []
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0)
    
    def test_christmas_tree(self):
        points = [(1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (3, 2), (1, 3), (2, 3)]
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0xFFF)
    
    def test_0x80_precise(self):
        points = [(1, 1)]
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0x80)
    def test_0x80_round(self):
        points = [(0.9, 1.1)]
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0x80)
    def test_0x80_round(self):
        points = [(1.1, 0.9)]
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0x80)
    def test_0x80_no_code(self):
        points = [(1.1, 0.8)]
        touchcode = touchcode_from_points(points)
        self.assertEqual(touchcode, 0)


class TestNormalization (TestCase):
    def test_normalize_to_simple_system(self):
        coords = (np.array([0,0]), np.array([3,0]), np.array([0,3]) )
        self.assertEqual(norm(coords, np.array([0, 0])), (0, 0))
        self.assertEqual(norm(coords, np.array([1, 0])), (1, 0))
        self.assertEqual(norm(coords, np.array([2, 0])), (2, 0))
        self.assertEqual(norm(coords, np.array([3, 0])), (3, 0))
        
        self.assertEqual(norm(coords, np.array([0, 1])), (0, 1))
        self.assertEqual(norm(coords, np.array([1, 1])), (1, 1))
        self.assertEqual(norm(coords, np.array([2, 1])), (2, 1))
        self.assertEqual(norm(coords, np.array([3, 1])), (3, 1))
        
        self.assertEqual(norm(coords, np.array([0, 2])), (0, 2))
        self.assertEqual(norm(coords, np.array([1, 2])), (1, 2))
        self.assertEqual(norm(coords, np.array([2, 2])), (2, 2))
        self.assertEqual(norm(coords, np.array([3, 2])), (3, 2))
        
        self.assertEqual(norm(coords, np.array([0, 3])), (0, 3))
        self.assertEqual(norm(coords, np.array([1, 3])), (1, 3))
        self.assertEqual(norm(coords, np.array([2, 3])), (2, 3))
        self.assertEqual(norm(coords, np.array([3, 3])), (3, 3))
               
    def test_normalize_to_rotated_system_90(self):
        coords = (np.array([3,3]), np.array([0,3]), np.array([3,0]) )
        self.assertEqual(norm(coords, np.array([0, 0])), (3, 3))
        self.assertEqual(norm(coords, np.array([1, 0])), (2, 3))
        self.assertEqual(norm(coords, np.array([2, 0])), (1, 3))
        self.assertEqual(norm(coords, np.array([3, 0])), (0, 3))
        
        self.assertEqual(norm(coords, np.array([0, 1])), (3, 2))
        self.assertEqual(norm(coords, np.array([1, 1])), (2, 2))
        self.assertEqual(norm(coords, np.array([2, 1])), (1, 2))
        self.assertEqual(norm(coords, np.array([3, 1])), (0, 2))
        
        self.assertEqual(norm(coords, np.array([0, 2])), (3, 1))
        self.assertEqual(norm(coords, np.array([1, 2])), (2, 1))
        self.assertEqual(norm(coords, np.array([2, 2])), (1, 1))
        self.assertEqual(norm(coords, np.array([3, 2])), (0, 1))
 
        self.assertEqual(norm(coords, np.array([0, 3])), (3, 0))
        self.assertEqual(norm(coords, np.array([1, 3])), (2, 0))
        self.assertEqual(norm(coords, np.array([2, 3])), (1, 0))
        self.assertEqual(norm(coords, np.array([3, 3])), (0, 0))
        
    def test_normalize_to_rotated_system_45(self):
        coords = (np.array([3,0]), np.array([3,3]), np.array([0,0]) )
        self.assertEqual(norm(coords, np.array([0, 0])), (0, 3))
        self.assertEqual(norm(coords, np.array([1, 0])), (0, 2))
        self.assertEqual(norm(coords, np.array([2, 0])), (0, 1))
        self.assertEqual(norm(coords, np.array([3, 0])), (0, 0))
        
        self.assertEqual(norm(coords, np.array([0, 1])), (1, 3))
        self.assertEqual(norm(coords, np.array([1, 1])), (1, 2))
        self.assertEqual(norm(coords, np.array([2, 1])), (1, 1))
        self.assertEqual(norm(coords, np.array([3, 1])), (1, 0))
        
        self.assertEqual(norm(coords, np.array([0, 2])), (2, 3))
        self.assertEqual(norm(coords, np.array([1, 2])), (2, 2))
        self.assertEqual(norm(coords, np.array([2, 2])), (2, 1))
        self.assertEqual(norm(coords, np.array([3, 2])), (2, 0))
        
        self.assertEqual(norm(coords, np.array([0, 3])), (3, 3))
        self.assertEqual(norm(coords, np.array([1, 3])), (3, 2))
        self.assertEqual(norm(coords, np.array([2, 3])), (3, 1))
        self.assertEqual(norm(coords, np.array([3, 3])), (3, 0))

    def test_with_samples_for_0x80(self):
        for sample in samples[0x80]:
            coords = np.array(sample)
            om = get_orientation_marks(coords)
            result = norm(om, sample[2])
            self.assertTrue(0.9 <= result[0] <= 1.1)
            self.assertTrue(0.9 <= result[1] <= 1.1)
        
class TestOrientationMarks (TestCase):
    def test_get_orientation_marks_simple (self):
        coords = np.array([ [0, 0], [3, 0], [0, 3], [1, 1] ])
        om = get_orientation_marks(coords)        
        self.assertTrue(np.array_equal(om[0], coords[0]))
        self.assertTrue(np.array_equal(om[1], coords[1]))
        self.assertTrue(np.array_equal(om[2], coords[2]))
        
    def test_get_orientation_marks_simple_different_order (self):
        coords = np.array([ [1, 1], [0, 3], [3, 0], [0, 0] ])
        om = get_orientation_marks(coords)      
        self.assertTrue(np.array_equal(om[0], coords[3]))
        self.assertTrue(np.array_equal(om[1], coords[2]))
        self.assertTrue(np.array_equal(om[2], coords[1]))
        
    def test_get_orientation_marks_simple_rotate_45(self):
        coords = np.array([ [0, 0], [3, 0], [2, 1], [3, 3] ])
        om = get_orientation_marks(coords)        
        self.assertTrue(np.array_equal(om[0], coords[1]))
        self.assertTrue(np.array_equal(om[1], coords[3]))
        self.assertTrue(np.array_equal(om[2], coords[0]))
        
    def test_get_orientation_marks_simple_rotate_90(self):
        coords = np.array([ [0, 3], [3, 0], [3, 3], [2, 2], [1, 1], [0, 1] ])
        om = get_orientation_marks(coords)        
        self.assertTrue(np.array_equal(om[0], coords[2]))
        self.assertTrue(np.array_equal(om[1], coords[0]))
        self.assertTrue(np.array_equal(om[2], coords[1]))
        
    def test_get_orientation_marks_simple_rotate_135(self):
        coords = np.array([ [0, 0], [0, 3], [3, 3], [1, 2], [1, 1] ])
        om = get_orientation_marks(coords)        
        self.assertTrue(np.array_equal(om[0], coords[1]))
        self.assertTrue(np.array_equal(om[1], coords[0]))
        self.assertTrue(np.array_equal(om[2], coords[2]))
        
    def test_get_orientation_marks_from_samples(self):
        for sample in samples[0x80]:
            coords = np.array(sample)
            om = get_orientation_marks(coords)        
            self.assertTrue(np.array_equal(om[0], coords[3]))
            self.assertTrue(np.array_equal(om[1], coords[1]))
            self.assertTrue(np.array_equal(om[2], coords[0]))

    def test_get_no_orientation_marks_on_garbage(self):
        coords = garbage[0]
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
        
    def test_get_no_orientation_marks_for_empty_points(self):
        coords = []
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
        
    def test_get_no_orientation_marks_for_non_coordinates(self):
        coords = [1,2,3]
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
        
    def test_get_no_orientation_marks_for_one_point(self):
        coords = np.array([ [1, 1] ])
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
        
    def test_get_no_orientation_marks_for_two_points(self):
        coords = np.array([ [1, 1], [1, 2] ])
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
    
    def test_get_no_orientation_marks_for_three_wrong_points(self):
        coords = np.array([ [0, 0], [3, 0], [0, 2] ])
        om = get_orientation_marks(coords)
        self.assertEqual(om, (None, None, None))
        

suite = unittest.TestLoader().loadTestsFromModule(TestOrientationMarks())
suite.addTests(unittest.TestLoader().loadTestsFromModule(TestNormalization()))
suite.addTests(unittest.TestLoader().loadTestsFromModule(TestPointsToTouchcode()))
unittest.TextTestRunner().run(suite)

...................F.
FAIL: test_christmas_tree (__main__.TestPointsToTouchcode)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-223-a8d5e70f8033>", line 13, in test_christmas_tree
    self.assertEqual(touchcode, 0xFFF)
AssertionError: 128 != 4095

----------------------------------------------------------------------
Ran 21 tests in 0.082s

FAILED (failures=1)


<unittest.runner.TextTestResult run=21 errors=0 failures=1>

In [167]:
"""Test Cell"""
import sys
import traceback
from termcolor import colored
import numpy as np
import io
from contextlib import redirect_stdout


for l in locals().copy():
    if l[0:5] == "test_":
        del locals()[l]

def run_all_tests(print_stdout = True, clean_tests = True):   
    tests = [test for test in globals() if test[0:5] == "test_"]
    tests.sort()
    
    print("+-----------------------+")
    print("|     Jupyter Tests     |")
    print("+-----------------------+\n")
    print(":: Loaded {0} Tests\n".format(len(tests)))
    
    errors = 0
    
    for test in tests:
        test_output = test_runner_output = ""
        
        with io.StringIO() as buf, redirect_stdout(buf):    
            try:
                globals()[test]()
                test_runner_output += colored('+ [Pass] {}'.format(test), 'green')
            except Exception as e:
                errors += 1
                _, exc_value, tb = sys.exc_info()
                filename, line, func, text = traceback.extract_tb(tb)[-1]
                if type(e).__name__ == "AssertionError":
                    test_runner_output += colored('- [Fail] {0} (Line {1}: {2})'.format(func, line, text), 'red')
                else:
                    test_runner_output += colored('- [Excp] {0} (Line {1}: {2})'.format(func, line, exc_value), 'red')
            test_output = buf.getvalue()
            
        print(test_runner_output)
        
        if(test_output and print_stdout):
            print("\n".join(list(map(lambda s: "\t" + s , test_output.strip().split("\n")))))
        
    print("\n{0} pass, {1} fail in {2} tests".format(len(tests) - errors, errors, len(tests)))

    list(map(lambda test: globals().pop(test), tests)) if clean_tests else 0
    
    return errors == 0
    
def test_get_orientation_vectors_simple():
    coords = np.array([ [0, 0], [3, 0], [0, 3], [1, 1] ])
    om = get_orientation_marks(coords)
    assert om[0] == coords[0]
    assert om[1] == coords[2]
    assert om[2] == coords[1]

def test_get_orientation_vectors_with_noise():
    coords = np.array([ [0, 0], [3, 0], [0, 3], [1, 1], [2, 1], [1, 2] ])
    assert False # implement
    
def test_x():
    x = 3 + 4
    print("hey!")
    assert x == 7

run_all_tests()


+-----------------------+
|     Jupyter Tests     |
+-----------------------+

:: Loaded 3 Tests

- [Fail] test_get_orientation_vectors_simple (Line 56: assert om[0] == coords[0])
	v1: [3 0], v2: [0 3], dst(v1, v2): 4.242640687119285]
	[[0 0]]
	Candidates for origin: [(0, 0), (0, 0)]
	origin: [[0 0]]
- [Fail] test_get_orientation_vectors_with_noise (Line 62: assert False # implement)
+ [Pass] test_x
	hey!

1 pass, 2 fail in 3 tests


c:\program files\python35\lib\site-packages\ipykernel\__main__.py:56: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


False